# Group 20 — Clustering

This notebook performs exploratory data analysis on customer and flights databases, covering data inspection, quality assessment, and preprocessing.

## Table of Contents
- [Data Import](#data-import)
- [Data Preprocessing](#preprocessing)

# <a id="data-import"></a> Data Import

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from scipy import stats
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Load the data

customer_db = pd.read_csv("data/DM_AIAI_CustomerDB.csv", index_col=0 )
flights_db = pd.read_csv("data/DM_AIAI_FlightsDB.csv")


# <a id="preprocessing"></a> Data Preprocessing

In this section we apply the preprocessing and feature engineering steps
described in the report:

- Fix data types and logical inconsistencies in **FlightsDB**
- Handle missing values and transform skewed variables in **CustomerDB**
- Create new features: log-transformed variables, points utilisation,
  cancellation flag, customer value score, flight activity score, and
  average flight distance.



In [57]:
import numpy as np

def preprocess_flights(flights_df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply preprocessing steps to the FlightsDB:
    - Convert YearMonthDate to datetime
    - Round down NumFlights and NumFlightsWithCompanions
    - Set DistanceKM = 0 where NumFlights == 0
    - Drop DollarCostPointsRedeemed
    - Add log-transformed versions of skewed variables
    - Create PointsUtilizationRatio = PointsRedeemed / PointsAccumulated
    """
    df = flights_df.copy()

    # 1. YearMonthDate -> datetime
    if 'YearMonthDate' in df.columns:
        df['YearMonthDate'] = pd.to_datetime(df['YearMonthDate'])

    # 2. Round down flight counts and cast to int
    for col in ['NumFlights', 'NumFlightsWithCompanions']:
        if col in df.columns:
            df[col] = np.floor(df[col]).astype(int)

    # 3. Fix logical inconsistency: DistanceKM must be 0 if NumFlights == 0
    if {'NumFlights', 'DistanceKM'}.issubset(df.columns):
        df.loc[df['NumFlights'] == 0, 'DistanceKM'] = 0

    # 4. Drop perfectly correlated variable
    if 'DollarCostPointsRedeemed' in df.columns:
        df = df.drop(columns=['DollarCostPointsRedeemed'])

    # 5. Log transforms for skewed numeric variables
    log_cols = ['DistanceKM', 'PointsAccumulated', 'PointsRedeemed']
    for col in log_cols:
        if col in df.columns:
            df[f'{col}_log'] = np.log1p(df[col])

    # 6. Points utilisation ratio
    if {'PointsRedeemed', 'PointsAccumulated'}.issubset(df.columns):
        denom = df['PointsAccumulated'].replace({0: np.nan})
        df['PointsUtilizationRatio'] = df['PointsRedeemed'] / denom

    return df


def preprocess_customers(customer_df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply preprocessing steps to the CustomerDB:
    - Create cancellation flag from CancellationDate
    - Group-median imputation (by LoyaltyStatus) for Income and Customer Lifetime Value
    - Log transform Customer Lifetime Value
    - Create Location feature (region) from Province or State (placeholder mapping)
    - Create CustomerValueScore composite feature
    """
    df = customer_df.copy()

    # 1. Cancellation flag
    if 'CancellationDate' in df.columns:
        df['CancelledFlag'] = df['CancellationDate'].notna().astype(int)

    # 2. Group-median imputation by LoyaltyStatus
    group_col = 'LoyaltyStatus'
    cols_to_impute = ['Income', 'Customer Lifetime Value']
    for col in cols_to_impute:
        if col in df.columns and group_col in df.columns:
            df[col] = df.groupby(group_col)[col].transform(
                lambda x: x.fillna(x.median())
            )

    # 3. Log transform Customer Lifetime Value (for variance stabilisation)
    if 'Customer Lifetime Value' in df.columns:
        df['CustomerLifetimeValue_log'] = np.log1p(df['Customer Lifetime Value'])

    # 4. Location feature (region mapping) – fill mapping as desired
    if 'Province or State' in df.columns:
        region_map = {
            'Ontario': 'Central',
            'Quebec': 'Quebec',
            'British Columbia': 'West',
            'Alberta': 'West',
            'Saskatchewan': 'West',
            'Manitoba': 'West',
            'New Brunswick': 'Atlantic',
            'Nova Scotia': 'Atlantic',
            'Prince Edward Island': 'Atlantic',
            'Newfoundland and Labrador': 'Atlantic',
            'Yukon': 'North',
            'Northwest Territories': 'North',
            'Nunavut': 'North',
        }

        df['Location'] = df['Province or State'].map(region_map).fillna('Other')

    # 5. Customer Value Score (simple composite of CLV and Income)
    clv_col = 'CustomerLifetimeValue_log'
    if clv_col in df.columns:
        clv_scaled = (df[clv_col] - df[clv_col].mean()) / df[clv_col].std(ddof=0)

        if 'Income' in df.columns:
            income_log = np.log1p(df['Income'].clip(lower=0))
            income_scaled = (income_log - income_log.mean()) / income_log.std(ddof=0)
            # Heavier weight on CLV, lighter on Income
            df['CustomerValueScore'] = 0.7 * clv_scaled + 0.3 * income_scaled
        else:
            df['CustomerValueScore'] = clv_scaled

    return df
 

In [58]:
def build_customer_flight_features(flights_df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate monthly flight records into customer-level features:
    - TotalFlights, TotalDistanceKM, TotalPointsAccumulated, TotalPointsRedeemed
    - MeanPointsUtilization
    - AverageFlightDistance
    - FlightActivityScore (based on z-scores of flights and distance)
    """
    id_col = 'Loyalty#'
    df = flights_df.copy()

    agg = (
        df
        .groupby(id_col)
        .agg(
            TotalFlights=('NumFlights', 'sum'),
            TotalDistanceKM=('DistanceKM', 'sum'),
            TotalPointsAccumulated=('PointsAccumulated', 'sum'),
            TotalPointsRedeemed=('PointsRedeemed', 'sum'),
            MeanPointsUtilization=('PointsUtilizationRatio', 'mean')
        )
        .reset_index()
    )

    # Average flight distance = total distance / total flights
    agg['AverageFlightDistance'] = agg['TotalDistanceKM'] / agg['TotalFlights'].replace({0: np.nan})

    # FlightActivityScore: combines total flights and distance (z-scores)
    for col in ['TotalFlights', 'TotalDistanceKM']:
        mean = agg[col].mean()
        std = agg[col].std(ddof=0)
        if std == 0:
            agg[f'{col}_z'] = 0
        else:
            agg[f'{col}_z'] = (agg[col] - mean) / std

    agg['FlightActivityScore'] = agg['TotalFlights_z'] + agg['TotalDistanceKM_z']

    return agg


In [59]:
# Run preprocessing for each table
customer_preprocessed = preprocess_customers(customer_db)
flights_preprocessed = preprocess_flights(flights_db)

# Build customer-level flight features
customer_flight_features = build_customer_flight_features(flights_preprocessed)

# Merge into a single modelling dataset (one row per customer)
id_col = 'Loyalty#'
model_df = (
    customer_preprocessed
    .merge(customer_flight_features, on=id_col, how='left')
)

print("Customer-preprocessed shape:", customer_preprocessed.shape)
print("Flights-preprocessed shape:", flights_preprocessed.shape)
print("Model dataset shape:", model_df.shape)

model_df.head()


Customer-preprocessed shape: (16921, 24)
Flights-preprocessed shape: (608436, 13)
Model dataset shape: (16921, 33)


,Loyalty#,First Name,Last Name,Customer Name,Country,Province or State,City,Latitude,Longitude,Postal code,...,CustomerValueScore,TotalFlights,TotalDistanceKM,TotalPointsAccumulated,TotalPointsRedeemed,MeanPointsUtilization,AverageFlightDistance,TotalFlights_z,TotalDistanceKM_z,FlightActivityScore
0,480934,Cecilia,Householder,Cecilia Householder,Canada,Ontario,Toronto,43.653225,-79.383186,M2Z 4K1,...,-0.326717,184.0,476229.8,50699.39,13517.9,0.267219,2588.205435,0.558399,1.228108,1.786507
1,549612,Dayle,Menez,Dayle Menez,Canada,Alberta,Edmonton,53.544388,-113.490930,T3G 6Y6,...,-1.043287,275.0,389938.9,42672.54,22457.8,1.559035,1417.959636,1.690882,0.693025,2.383908
2,429460,Necole,Hannon,Necole Hannon,Canada,British Columbia,Vancouver,49.282730,-123.120740,V6E 3D9,...,-1.043248,116.0,208146.7,23832.41,5479.6,0.947354,1794.368103,-0.287852,-0.434252,-0.722105
3,608370,Queen,Hagee,Queen Hagee,Canada,Ontario,Toronto,43.653225,-79.383186,P1W 1K4,...,-1.043248,191.0,372178.3,38595.63,16331.5,1.171049,1948.577487,0.645513,0.582893,1.228406
4,530508,Claire,Latting,Claire Latting,Canada,Quebec,Hull,45.428730,-75.713364,J8Y 3Z5,...,-0.304330,167.0,344036.5,36916.56,0.0,0.000000,2060.098802,0.346836,0.408389,0.755225


In [60]:

def winsorize_dataframe(df, columns, limits=(0.01, 0.01)):
    """
    Apply winsorization to each column in `columns`.
    limits=(lower_pct, upper_pct) means: cap values at the 1st and 99th percentile.

    Returns the winsorized copy of df.
    """
    df = df.copy()
    for col in columns:
        if col in df.columns:
            # winsorize returns masked arrays -> convert to normal array
            df[col] = winsorize(df[col], limits=limits).data
    return df


In [61]:
flight_outlier_cols = [
    'TotalFlights',
    'TotalDistanceKM',
    'TotalPointsAccumulated',
    'TotalPointsRedeemed',
    'AverageFlightDistance'
]

customer_flight_features_wins = winsorize_dataframe(
    customer_flight_features,
    columns=flight_outlier_cols,
    limits=(0.01, 0.01)    # winsorize at 1% and 99%
)


In [62]:
customer_outlier_cols = [
    'Income',
    'Customer Lifetime Value',
    'CustomerLifetimeValue_log'
]

customer_preprocessed_wins = winsorize_dataframe(
    customer_preprocessed,
    columns=customer_outlier_cols,
    limits=(0.01, 0.01)
)


In [63]:
id_col = 'Loyalty#'

model_df = (
    customer_preprocessed_wins
    .merge(customer_flight_features_wins, on=id_col, how='left')
)

print("Final model_df shape:", model_df.shape)
model_df.head()


Final model_df shape: (16921, 33)


,Loyalty#,First Name,Last Name,Customer Name,Country,Province or State,City,Latitude,Longitude,Postal code,...,CustomerValueScore,TotalFlights,TotalDistanceKM,TotalPointsAccumulated,TotalPointsRedeemed,MeanPointsUtilization,AverageFlightDistance,TotalFlights_z,TotalDistanceKM_z,FlightActivityScore
0,480934,Cecilia,Householder,Cecilia Householder,Canada,Ontario,Toronto,43.653225,-79.383186,M2Z 4K1,...,-0.326717,184.0,476229.8,50699.39,13517.9,0.267219,2588.205435,0.558399,1.228108,1.786507
1,549612,Dayle,Menez,Dayle Menez,Canada,Alberta,Edmonton,53.544388,-113.490930,T3G 6Y6,...,-1.043287,275.0,389938.9,42672.54,22457.8,1.559035,1417.959636,1.690882,0.693025,2.383908
2,429460,Necole,Hannon,Necole Hannon,Canada,British Columbia,Vancouver,49.282730,-123.120740,V6E 3D9,...,-1.043248,116.0,208146.7,23832.41,5479.6,0.947354,1794.368103,-0.287852,-0.434252,-0.722105
3,608370,Queen,Hagee,Queen Hagee,Canada,Ontario,Toronto,43.653225,-79.383186,P1W 1K4,...,-1.043248,191.0,372178.3,38595.63,16331.5,1.171049,1948.577487,0.645513,0.582893,1.228406
4,530508,Claire,Latting,Claire Latting,Canada,Quebec,Hull,45.428730,-75.713364,J8Y 3Z5,...,-0.304330,167.0,344036.5,36916.56,0.0,0.000000,2060.098802,0.346836,0.408389,0.755225


In [64]:
# Check for remaining missing values
model_df.isna().sum().sort_values(ascending=False).head(15)


CancellationDate          14611
AverageFlightDistance      1549
MeanPointsUtilization      1545
FlightActivityScore          20
TotalPointsRedeemed          20
TotalFlights                 20
TotalFlights_z               20
TotalPointsAccumulated       20
TotalDistanceKM              20
TotalDistanceKM_z            20
Loyalty#                      0
First Name                    0
Last Name                     0
Province or State             0
City                          0
dtype: int64

In [65]:
# Drop CancellationDate (flag already exists)
model_df = model_df.drop(columns=['CancellationDate'], errors='ignore')

# Fill average distance for zero-flight customers
model_df['AverageFlightDistance'] = model_df['AverageFlightDistance'].fillna(0)

# Fill points utilization for customers with no point activity
model_df['MeanPointsUtilization'] = model_df['MeanPointsUtilization'].fillna(0)

# Fill ALL flight-related NaNs with 0
flight_cols = [
    'TotalFlights', 'TotalDistanceKM', 'TotalPointsAccumulated',
    'TotalPointsRedeemed', 'MeanPointsUtilization', 'AverageFlightDistance',
    'TotalFlights_z', 'TotalDistanceKM_z', 'FlightActivityScore'
]

for col in flight_cols:
    model_df[col] = model_df[col].fillna(0)


In [66]:
# Numerical features for clustering
numeric_features = [
    'Income',
    'CustomerLifetimeValue_log',
    'CustomerValueScore',
    
    'TotalFlights',
    'TotalDistanceKM',
    'AverageFlightDistance',
    'TotalPointsAccumulated',
    'TotalPointsRedeemed',
    'MeanPointsUtilization',
    'FlightActivityScore',

    'CancelledFlag'
]

# Categorical features to encode
categorical_features = [
    'EnrollmentType',  # Bronze, Silver, Gold ...
    'Location'         # Region/Province after mapping
]


In [67]:
df_selected = model_df[numeric_features + categorical_features].copy()
df_selected.head()


,Income,CustomerLifetimeValue_log,CustomerValueScore,TotalFlights,TotalDistanceKM,AverageFlightDistance,TotalPointsAccumulated,TotalPointsRedeemed,MeanPointsUtilization,FlightActivityScore,CancelledFlag,EnrollmentType,Location
0,70146.0,8.253264,-0.326717,184.0,476229.8,2588.205435,50699.39,13517.9,0.267219,1.786507,0,Standard,Central
1,0.0,8.253386,-1.043287,275.0,389938.9,1417.959636,42672.54,22457.8,1.559035,2.383908,0,Standard,West
2,0.0,8.253423,-1.043248,116.0,208146.7,1794.368103,23832.41,5479.6,0.947354,-0.722105,1,Standard,West
3,0.0,8.253423,-1.043248,191.0,372178.3,1948.577487,38595.63,16331.5,1.171049,1.228406,0,Standard,Central
4,97832.0,8.254214,-0.304330,167.0,344036.5,2060.098802,36916.56,0.0,0.000000,0.755225,0,2021 Promotion,Quebec


In [68]:
df_encoded = pd.get_dummies(df_selected, columns=categorical_features, drop_first=True)
df_encoded.head()


,Income,CustomerLifetimeValue_log,CustomerValueScore,TotalFlights,TotalDistanceKM,AverageFlightDistance,TotalPointsAccumulated,TotalPointsRedeemed,MeanPointsUtilization,FlightActivityScore,CancelledFlag,EnrollmentType_Standard,Location_Central,Location_North,Location_Other,Location_Quebec,Location_West
0,70146.0,8.253264,-0.326717,184.0,476229.8,2588.205435,50699.39,13517.9,0.267219,1.786507,0,True,True,False,False,False,False
1,0.0,8.253386,-1.043287,275.0,389938.9,1417.959636,42672.54,22457.8,1.559035,2.383908,0,True,False,False,False,False,True
2,0.0,8.253423,-1.043248,116.0,208146.7,1794.368103,23832.41,5479.6,0.947354,-0.722105,1,True,False,False,False,False,True
3,0.0,8.253423,-1.043248,191.0,372178.3,1948.577487,38595.63,16331.5,1.171049,1.228406,0,True,True,False,False,False,False
4,97832.0,8.254214,-0.304330,167.0,344036.5,2060.098802,36916.56,0.0,0.000000,0.755225,0,False,False,False,False,True,False


In [69]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_encoded)

X_scaled[:5]


array([[ 1.06862073, -0.76270952, -0.42154618,  0.56623022,  1.24229357,
         0.59190799,  1.32410286,  0.58217805, -0.25048664,  0.88569868,
        -0.39761798,  0.26978251,  1.44725606, -0.0816278 , -0.12443234,
        -0.49323119, -0.79095667],
       [-1.24436294, -0.76251978, -1.34609924,  1.70616109,  0.70299251,
        -0.43469811,  0.83925221,  1.6234558 ,  0.61888987,  1.18482616,
        -0.39761798,  0.26978251, -0.69096273, -0.0816278 , -0.12443234,
        -0.49323119,  1.26429176],
       [-1.24436294, -0.76246326, -1.34604875, -0.28558625, -0.43317279,
        -0.10449122, -0.29875948, -0.35408554,  0.20723573, -0.37040044,
         2.51497678,  0.26978251, -0.69096273, -0.0816278 , -0.12443234,
        -0.49323119,  1.26429176],
       [-1.24436294, -0.76246326, -1.34604875,  0.65391721,  0.59199227,
         0.03079002,  0.59299217,  0.90989303,  0.35778005,  0.60624955,
        -0.39761798,  0.26978251,  1.44725606, -0.0816278 , -0.12443234,
        -0.49323119

# DBSCAN 


To identify anomalous customer behavior, the DBSCAN algorithm was applied to the scaled feature space. In this project, DBSCAN is used exclusively for outlier detection, not for final customer segmentation.

In [70]:
dbscan = DBSCAN(eps=1.9, min_samples=20, n_jobs=1)
dbscan_labels = dbscan.fit_predict(X_scaled)

outlier_count = Counter(dbscan_labels)
print(f"DBSCAN results: {outlier_count}")
print(f"Outliers detected: {outlier_count.get(-1, 0)}")
print(f"Core customers: {outlier_count.get(0, 0)}")

DBSCAN results: Counter({np.int64(1): 5151, np.int64(0): 4323, np.int64(3): 2573, np.int64(6): 985, np.int64(-1): 848, np.int64(2): 797, np.int64(4): 673, np.int64(9): 384, np.int64(5): 363, np.int64(7): 285, np.int64(10): 187, np.int64(8): 150, np.int64(11): 89, np.int64(12): 63, np.int64(13): 30, np.int64(14): 20})
Outliers detected: 848
Core customers: 4323


The algorithm identified multiple dense regions in the data, reflecting local behavioral patterns across customers. A small fraction of observations was classified as noise.

In [71]:
core_mask = (dbscan_labels != -1)

X_scaled_core = X_scaled[core_mask]

print(f"Core customers kept: {len(X_scaled_core):,}")

Core customers kept: 16,073


DBSCAN successfully identified a small proportion of anomalous customers (~5%), while preserving the vast majority of the dataset for subsequent analysis. This confirms that most customers exhibit consistent behavior, and that outliers can be safely removed to improve the robustness of downstream clustering and modeling tasks.

## Clustering

In [72]:
inertias = []
silhouettes = []
K = range(2, 11)

for k in K:
    kmeans = KMeans(
        n_clusters=k,
        init='k-means++',
        random_state=42,
        n_init=20
    )
    labels = kmeans.fit_predict(X_scaled_core)
    
    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X_scaled_core, labels))